#### source: https://github.com/google-research/bert/blob/master/README.md

### BERT fine-tune 實作分成三部分
### 1. 準備資料
### 2. 修改run_classifier.py 中的processor
### 3. 執行run_classifier.py 與設定參數

# ---------------------------------------------------

#### 1. 準備資料

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle
import uuid
from pandas.core.frame import DataFrame

In [2]:
df = pd.read_csv('./data/19999_question_category_a_v13.csv')

In [3]:
# df=df[((df.category_a_target != 7) & (df.category_a_target != 1) 
#        & (df.category_a_target != 2) & (df.category_a_target != 3))]

#df=df[((df.category_a_target != 7))]

In [4]:
df.head(2)

,description,category_a,category_b,category_c,category_d,category,description_clean,category_target
0,"無法會員登入,顯示訊息:無效的帳號或使用者不存在。帳號:96122401",CAMP,CAMP,CAMP,CAMP,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,因今天有事請假，要補登先前的加班，但開啟camp後，出勤表單顯示的內容看起來像測試的，沒有最...,CAMP,CAMP,CAMP,CAMP,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4


In [5]:
df.shape

(11159, 8)

In [6]:
df.groupby(['category','category_target']).size()

category    category_target
Outlook與郵件  0                  2178
SAP         1                  1443
其他          2                  3521
網路          3                  2631
資訊系統        4                  1386
dtype: int64

In [7]:
#drop category
# df=df.loc[(df['category_target'] != 2) & (df['category_target'] != 7)]

In [8]:
df=df.drop(['category_a','category_b','category_c','category_d','description'], axis=1)
df.head(5)

,category,description_clean,category_target
0,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4
2,資訊系統,簽核BPM有問題 都顯示亂碼,4
3,資訊系統,無法在上面使用CAMP進行表單簽核動作,4
4,資訊系統,出差同仁無法登入camp 委請同事詢問,4


In [9]:
df.columns = ['category_a','description','category_a_target']

In [10]:
df.head(5)

,category_a,description,category_a_target
0,資訊系統,無法會員登入 顯示訊息 無效的帳號或不存在 帳號,4
1,資訊系統,因有事請假 要補登先前的加班 但開啟camp後 出勤表單顯示的內容像測試的 沒有的時間也沒辦...,4
2,資訊系統,簽核BPM有問題 都顯示亂碼,4
3,資訊系統,無法在上面使用CAMP進行表單簽核動作,4
4,資訊系統,出差同仁無法登入camp 委請同事詢問,4


In [11]:
#結果放到run_classifier.py get_labels 443行
sorted(df.category_a_target.unique())

[0, 1, 2, 3, 4]

In [12]:
pd.unique(df[['category_a', 'category_a_target']].values.ravel('K'))

array(['資訊系統', 'SAP', '其他', '網路', 'Outlook與郵件', 4, 1, 2, 3, 0],
      dtype=object)

In [13]:
pd.concat([df['category_a'], df['category_a_target']]).unique()

array(['資訊系統', 'SAP', '其他', '網路', 'Outlook與郵件', 4, 1, 2, 3, 0],
      dtype=object)

In [14]:
category_df =df[['category_a', 'category_a_target']]
category_df = category_df.drop_duplicates()
category_df

,category_a,category_a_target
0,資訊系統,4
14,SAP,1
97,其他,2
544,網路,3
602,Outlook與郵件,0


In [15]:
df.shape

(11159, 3)

In [16]:
df.groupby(['category_a','category_a_target']).size()

category_a  category_a_target
Outlook與郵件  0                    2178
SAP         1                    1443
其他          2                    3521
網路          3                    2631
資訊系統        4                    1386
dtype: int64

### 現在我們將資料轉換成 BERT 需要的資料形式。根據官方fine tune教學，需要給的資料有四個column，分別是:
### (1) guid: Unique id for the example.

### (2) text_a: text_a: string. The untokenized text of the first sequence. For single sequence tasks, only this sequence must be specified.
### (3) text_b(Optional): string. The untokenized text of the second sequence. Only must be specified for sequence pair tasks.
### (4) label(Optional): string. The label of the example. This should be specified for train and dev examples, but not for test examples.

In [17]:
data_for_bert = []
strat_list = []
for i in range(len(df)):
    data_for_bert.append((i, #guid
                          df.iloc[i,:].description, #text_a
                          None, #text_b
                          df.iloc[i,:].category_a_target #label
                         ))
    strat_list.append((i,df.iloc[i,:].category_a_target))

In [18]:
len(data_for_bert)

11159

### 產生test.csv,下列兩項擇一

In [19]:
# #generate t_cest data from 19999_test_question_v1.csv
# test_df = pd.read_csv('./data/19999_test_question_v2.csv')
# test_df=test_df.drop(['category_a','category_a_b','description_clean'], axis=1)
# test_df
# test_df.columns = ['description','label']
# test_df.to_csv('tmp/call_center_input/test.csv', index=False, header=True)

In [20]:
#random select to test.csv
#import random
#test_df = pd.DataFrame(random.sample(data_for_bert, 10))
test_df = pd.DataFrame(data_for_bert)
test_df=test_df.sample(frac=0.03) #frac是要返回的比例
test_df.columns = ['id', 'description', 'text_b', 'label']
test_df_2=test_df.drop(['id', 'text_b'], axis=1)
#test_df
test_df_2.to_csv('tmp/call_center_input/test.csv', index=False, header=True)

### 產生test.tsv

In [21]:
#test.csv -> test.tsv
import pandas as pd
df_temp = pd.read_csv('tmp/call_center_input/test.csv') 
df_temp.to_csv('tmp/call_center_input/test.tsv', sep='\t', index=False, header=True) #

In [22]:
#test_df.head(5)
len(test_df)

335

In [23]:
len(data_for_bert)

11159

In [24]:
for s in test_df.id:
    data_for_bert=[k for k in data_for_bert if s not in k]

In [25]:
len(data_for_bert)

10824

In [26]:
for s in test_df.id:
    strat_list=[k for k in strat_list if s not in k]

In [27]:
len(strat_list)

10824

In [28]:
strat_list_temp = []
for s in strat_list:
    strat_list_temp.append(s[1])
strat_list=strat_list_temp

### 產生Train.tsv

In [29]:
train_df = pd.DataFrame(data_for_bert)
train_df.columns = ['id', 'description', 'text_b', 'label']
train_df_2=train_df.drop(['id', 'text_b'], axis=1)
#test_df
train_df_2.to_csv('tmp/call_center_input/train.csv', index=False, header=True)

In [30]:
#test.csv -> test.tsv
import pandas as pd
df_temp = pd.read_csv('tmp/call_center_input/train.csv') 
df_temp.to_csv('tmp/call_center_input/train.tsv', sep='\t', index=False, header=True) #

### 產生pickle_file

In [31]:
train, dev = train_test_split(data_for_bert, test_size=0.2, stratify=strat_list)

In [32]:
print('length for fine-tune training data: ',len(train))
print('length for fine-tune development data: ',len(dev))
print('first instance:', data_for_bert[0])

length for fine-tune training data:  8659
length for fine-tune development data:  2165
first instance: (0, '無法會員登入 顯示訊息 無效的帳號或不存在 帳號', None, 4)


In [33]:
with open('bert_train.p',mode = 'wb') as pickle_file:
    pickle.dump(train, pickle_file)

with open('bert_dev.p',mode = 'wb') as pickle_file:
    pickle.dump(dev, pickle_file)


#### 2. 於影片中說明如何修改 run_classifier.py 中的 processor

#### 3. 執行run_classifier.py 並設定參數

In [34]:
!rm -r tmp/call_center_output/

rm: cannot remove 'tmp/call_center_output/': No such file or directory


In [35]:
!python run_classifier.py \
  --task_name=call_center \
  --do_train=true \
  --do_eval=true \
  --data_dir=. \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint= chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --output_dir=tmp/call_center_output/ \
  --train_batch_size=32 \ #16, 32
  --learning_rate=2e-5 \  #5e-5, 3e-5, 2e-5
  --num_train_epochs=3.0 \#3, 4

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_model_dir': 'tmp/call_center_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7908e100b8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=

### predict test.tsv

In [36]:
!python run_classifier.py \
  --task_name=call_center \
  --do_predict=true \
  --data_dir=tmp/call_center_input \
  --vocab_file=chinese_L-12_H-768_A-12/vocab.txt \
  --bert_config_file=chinese_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint=chinese_L-12_H-768_A-12/bert_model.ckpt \
  --max_seq_length=128 \
  --output_dir=tmp/call_center_output/

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using config: {'_model_dir': 'tmp/call_center_output/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb126201550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=

### 預測準確率

In [37]:
#test_df
#test_df2 = pd.read_csv('tmp/call_center_input/test.csv')
test_label = test_df['label'].tolist()
len(test_label)

335

In [38]:
test_predict_df = pd.read_csv('tmp/call_center_output/test_results.tsv',header=None,sep = '\t')
#test_predict_df

In [39]:
# a=np.argmax(test_predict_df.as_matrix(), axis=-1)  
# a

In [40]:
test_predict_label =np.argmax(test_predict_df.as_matrix(), axis=-1)  
#test_predict_label =a

result = [1 if x==y else 0 for x,y in zip(test_label,test_predict_label)]


sum(result)/len(result)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


0.8507462686567164

In [41]:
# category_df[category_df['category_a_target']==8].category_a.iloc[0]

In [42]:
#test_predict_label[0]
predict_no = []
for i in range(len(test_predict_label)):
    if not (test_predict_label[i]==test_label[i]):
        predict_no.append((i,
                          #test_predict_label[i],
                          category_df[category_df['category_a_target']==test_predict_label[i]].category_a.iloc[0],
                          #test_label[i],
                          category_df[category_df['category_a_target']==test_label[i]].category_a.iloc[0],
                          test_df.iloc[i].description
                         ))
df_predict_no=DataFrame(predict_no)
df_predict_no.columns = ['id','predit','label','description']
df_predict_no

,id,predit,label,description
0,3,網路,其他,原本是兩個公司帳號 造成使用公司內部網頁都需要重新輸入帳號密碼 目前已恢復為一個公司別 但需...
1,4,網路,其他,透過網路安裝WIN 7時 出現不支援此lan 的錯誤訊息
2,20,網路,資訊系統,於MyQuanta首頁點選左上角出勤資訊後畫面跳出異常 無法連線至該網頁
3,21,網路,其他,國家的客人要Call進公司的號碼
4,35,其他,網路,NB 無法使用 WLAN
5,36,網路,其他,因工作需求有時需要連線redhat ftp下載測試images 目前無法連線 ftp /...
6,40,網路,其他,我人在QSMC 但使用的domain是quanta 1. 登入win時都須先把網路拔掉 ...
7,41,網路,資訊系統,南棟警衛反應無法使用
8,55,其他,網路,登入後速度很慢 無法開啟伺服器檔案
9,56,網路,其他,我的位置上的 Cisco 無法開啟 網路線 接NB 有網路 Cisco 跟網路線 ...


In [43]:
#t_df=df.loc[df['description']=='人在QCA出差 密碼過期無法收發Mail']

# t_df=df.loc[df['description']=='ProE 在使用  很容易被踢出  不確定是否是License 問題  或多人在使用']
# t_df

In [44]:
print(len(test_predict_label))
print(len(test_label))

335
335


In [45]:
# print(test_predict_label)
# print(test_label)